## Sign Language Classification

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
import os 
import cv2

In [3]:
DATA_DIR = "./data"

In [4]:
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

In [5]:
no_of_classes = 26
dataset_size = 50

## Capturing Images

In [6]:
cap = cv2.VideoCapture(0)
for j in range(no_of_classes):
    if not os.path.exists(os.path.join(DATA_DIR, str(j))):
        os.makedirs(os.path.join(DATA_DIR, str(j)))

    print(f"Collecting Data for class {j}")

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, "Ready? Press 'Q'" , (100,50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow("frame", frame)
        
        if cv2.waitKey(25) == ord("q"):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        cv2.imshow("frame", frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(j), f"{counter}.jpg"), frame)

        counter += 1

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

## Creating Dataset from the images collected

In [1]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.
